# Surface Salinity PCA 

After we have proved that salinity is a good tracer for total nitrate, including the part consumed by phytoplankton, we can now begin our analysis on surface salinity, which is a good indicator for both nitrate and see water movement.

## PCA 2011&2018 Summer

In order to make sure that our PCA will not mix different modes, I intend to conduct PCA on summers of 2011 and 2018 and check if they share PCs and Modes which have the same physical explanation.

In [1]:
# SSS_PCA_Comparison_2011_vs_2018.py
# Performs independent PCA on Sea Surface Salinity (SSS) for:
# 1. Summer 2011 (May 1 - Aug 31)
# 2. Summer 2018 (May 1 - Aug 31)
# And generates side-by-side comparison plots to check for consistency.

import netCDF4 as nc
import numpy as np
import datetime
import os
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 

# --- 1. Configuration ---

# !!! 请根据实际环境修改 !!!
BASE_DIR = '/results2/SalishSea/nowcast-green.201905/' 
FNAME_HEAD = 'SalishSea_1d_'
FNAME_TAIL = '_grid_T.nc' 

# 输出目录
OUT_DIR = '/home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
    print(f"Created output directory: {OUT_DIR}")

# 变量设置
VAR_SALINITY = 'vosaline'
VAR_LAT = 'nav_lat'
VAR_LON = 'nav_lon'

# 分析参数
YEARS_TO_COMPARE = [2011, 2018]
START_MONTH, START_DAY = 5, 1   # May 1st
END_MONTH, END_DAY = 8, 31      # Aug 31st

SPATIAL_SUBSAMPLE_STEP = 8 
N_COMPONENTS = 3  # 只对比前3个模态通常就够了

# --- 2. Data Loading Helper (Robust Land Masking) ---

def get_file_path(date_obj):
    indir_date = date_obj.strftime('%d%b%y').lower() + '/'
    fname_date = date_obj.strftime('%Y%m%d')
    return os.path.join(BASE_DIR, indir_date, FNAME_HEAD + fname_date + '_' + fname_date + FNAME_TAIL)

def load_daily_sss(date_obj, step):
    """
    Loads one day of SSS data.
    Returns: (sss_flat_clean, lon_2d, lat_2d, valid_mask_1d)
    """
    fpath = get_file_path(date_obj)
    if not os.path.exists(fpath):
        return None, None, None, None
    
    try:
        with nc.Dataset(fpath, 'r') as ncfile:
            sss = ncfile.variables[VAR_SALINITY][0, 0, ::step, ::step]
            lon = ncfile.variables[VAR_LON][::step, ::step]
            lat = ncfile.variables[VAR_LAT][::step, ::step]
            
            # Masking: Lon != 0 AND SSS is not NaN
            mask_valid = (lon != 0.0) & (~np.isnan(sss)) & (sss < 1000) # Simple range check
            
            # Flatten everything
            sss_flat = sss.flatten()
            mask_flat = mask_valid.flatten()
            
            # Apply mask
            sss_clean = sss_flat[mask_flat]
            
            return sss_clean, lon, lat, mask_flat
    except Exception as e:
        print(f"Error loading {date_obj}: {e}")
        return None, None, None, None

# --- 3. Single Year Analysis Engine ---

def analyze_year(year):
    """
    Runs PCA for a single year's summer period.
    Returns a dictionary with all results needed for plotting.
    """
    print(f"\n--- Analyzing Year {year} (May 1 - Aug 31) ---")
    start_date = datetime.date(year, START_MONTH, START_DAY)
    end_date = datetime.date(year, END_MONTH, END_DAY)
    
    data_list = []
    dates_list = []
    
    # Static grid info (captured from first valid day)
    grid_lon, grid_lat, grid_mask = None, None, None
    original_shape = None

    curr = start_date
    while curr <= end_date:
        vec, lon, lat, mask = load_daily_sss(curr, SPATIAL_SUBSAMPLE_STEP)
        if vec is not None:
            if grid_lon is None:
                grid_lon, grid_lat, grid_mask = lon, lat, mask
                original_shape = lon.shape
            
            # Consistency check
            if vec.shape[0] != grid_mask.sum():
                print(f"  Skipping {curr}: Grid mismatch (Mask len {grid_mask.sum()} vs Data {vec.shape[0]})")
            else:
                data_list.append(vec)
                dates_list.append(curr)
        curr += datetime.timedelta(days=1)
        
    if not data_list:
        return None
    
    # PCA Execution
    X = np.array(data_list)
    print(f"  Data shape: {X.shape}")
    
    pca = PCA(n_components=N_COMPONENTS)
    # Fit on centered data
    pcs = pca.fit_transform(X - X.mean(axis=0)) 
    eofs = pca.components_
    var_ratio = pca.explained_variance_ratio_
    
    # --- Auto-Flip Signs for Consistency ---
    # PCA signs are arbitrary. We flip them so the maximum absolute value is always positive.
    # This makes comparing maps between years much easier.
    for i in range(N_COMPONENTS):
        max_idx = np.argmax(np.abs(eofs[i]))
        if eofs[i][max_idx] < 0:
            eofs[i] *= -1
            pcs[:, i] *= -1 # Must flip PC time series too!
            
    return {
        'year': year,
        'pcs': pcs, # (Time, Modes)
        'eofs': eofs, # (Modes, Space)
        'var': var_ratio,
        'dates': dates_list,
        'lon': grid_lon,
        'lat': grid_lat,
        'mask': grid_mask,
        'shape': original_shape
    }

# --- 4. Comparison Plotting Functions ---

def plot_side_by_side_spatial(res1, res2, mode_idx):
    """Plots the Spatial Mode (EOF) of Year 1 and Year 2 side by side."""
    
    fig = plt.figure(figsize=(14, 7))
    
    # Shared settings
    extent = [-125.5, -122, 48, 50.5]
    levels = np.linspace(-0.15, 0.15, 41) # Fixed scale for easier comparison!
    # Note: You might need to adjust levels if EOF values are very small/large
    
    results_list = [res1, res2]
    
    for k in range(2):
        res = results_list[k]
        ax = fig.add_subplot(1, 2, k+1, projection=ccrs.Mercator(central_longitude=-124))
        ax.set_extent(extent, crs=ccrs.PlateCarree())
        
        # Reconstruct 2D field
        eof_1d = res['eofs'][mode_idx]
        eof_2d_flat = np.full(res['mask'].shape, np.nan)
        eof_2d_flat[res['mask']] = eof_1d
        eof_2d = eof_2d_flat.reshape(res['shape'])
        
        # Adaptive Color Limits (override fixed levels if needed)
        limit = np.nanmax(np.abs(eof_2d))
        current_levels = np.linspace(-limit, limit, 40)
        
        cf = ax.contourf(res['lon'], res['lat'], eof_2d, 
                         levels=current_levels, cmap='RdBu_r', 
                         transform=ccrs.PlateCarree(), extend='both')
        
        ax.add_feature(cfeature.LAND, color='lightgray', zorder=2)
        ax.add_feature(cfeature.COASTLINE, linewidth=0.8, zorder=3)
        
        ax.set_title(f"{res['year']} EOF-{mode_idx+1}\n(Explains {res['var'][mode_idx]*100:.1f}%)", fontsize=14)
        plt.colorbar(cf, ax=ax, shrink=0.8, orientation='horizontal', pad=0.05)

    plt.suptitle(f"Comparison of Spatial Mode {mode_idx+1} (Summer)", fontsize=16)
    
    save_path = os.path.join(OUT_DIR, f"Compare_Spatial_Mode_{mode_idx+1}.png")
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    plt.close()
    print(f"Saved comparison map: {save_path}")

def plot_overlaid_timeseries(res1, res2, mode_idx):
    """Plots PC time series overlaid on a Day-of-Year axis."""
    
    plt.figure(figsize=(12, 4))
    
    # Helper to get day of year
    def get_doy(dates):
        return [d.timetuple().tm_yday for d in dates]
    
    doy1 = get_doy(res1['dates'])
    pc1 = res1['pcs'][:, mode_idx]
    
    doy2 = get_doy(res2['dates'])
    pc2 = res2['pcs'][:, mode_idx]
    
    plt.plot(doy1, pc1, 'r-', lw=2, alpha=0.8, label=f"{res1['year']} (Var: {res1['var'][mode_idx]*100:.1f}%)")
    plt.plot(doy2, pc2, 'b--', lw=2, alpha=0.8, label=f"{res2['year']} (Var: {res2['var'][mode_idx]*100:.1f}%)")
    
    plt.xlabel("Day of Year (May 1 is approx Day 121)")
    plt.ylabel("PC Amplitude")
    plt.title(f"PC-{mode_idx+1} Time Series Comparison")
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.legend()
    
    save_path = os.path.join(OUT_DIR, f"Compare_TimeSeries_PC_{mode_idx+1}.svg")
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    plt.close()
    print(f"Saved comparison TS: {save_path}")

# --- 5. Main Execution ---

if __name__ == "__main__":
    # 1. Analyze 2011
    res_2011 = analyze_year(2011)
    
    # 2. Analyze 2018
    res_2018 = analyze_year(2018)
    
    if res_2011 and res_2018:
        print("\n--- Generating Comparison Plots ---")
        
        # Compare first 3 modes
        for i in range(N_COMPONENTS):
            # A. Spatial Comparison
            plot_side_by_side_spatial(res_2011, res_2018, mode_idx=i)
            
            # B. Time Series Comparison
            plot_overlaid_timeseries(res_2011, res_2018, mode_idx=i)
            
        print(f"\nDone! Check folder: {OUT_DIR}")
        print("Tip: If EOF-1 maps look similar (e.g., both river plume patterns), you can safely combine years.")
    else:
        print("Error: Could not process both years successfully.")


--- Analyzing Year 2011 (May 1 - Aug 31) ---
  Data shape: (123, 2880)

--- Analyzing Year 2018 (May 1 - Aug 31) ---
  Data shape: (123, 2880)

--- Generating Comparison Plots ---
Saved comparison map: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA/Compare_Spatial_Mode_1.png
Saved comparison TS: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA/Compare_TimeSeries_PC_1.svg
Saved comparison map: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA/Compare_Spatial_Mode_2.png
Saved comparison TS: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA/Compare_TimeSeries_PC_2.svg
Saved comparison map: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA/Compare_Spatial_Mode_3.png
Saved comparison TS: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA/Compare_TimeSeries_PC_3.svg

Done! Ch

The 2 yr case was a little weird. Mode 2 can be explained as Wind-Driven Plume Steering, or the turning of Fraser Plume caused by wind. Mode 3 was explained as Cross-Channel / Upwelling. We can use 10yr-summer daily data to verify it.

## PCA on 10-year Summers (2010-2019)

In [2]:
# SSS_PCA_Decade_Calc.py
# 1. Loads SSS for Summer (May 1 - Aug 31) from 2010 to 2019.
# 2. Performs PCA (EOF analysis).
# 3. Saves PCs to CSV and all matrices to .npz for fast plotting.

import netCDF4 as nc
import numpy as np
import datetime
import os
import pandas as pd
from sklearn.decomposition import PCA

# --- Configuration ---
# !!! 修改路径 !!!
BASE_DIR = '/results2/SalishSea/nowcast-green.201905/' 
FNAME_HEAD = 'SalishSea_1d_'
FNAME_TAIL = '_grid_T.nc' 
OUT_DIR = '/home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/'

if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

# 时间范围: 2010-2019, 每年 5.1 - 8.31
YEARS = range(2010, 2020) 
START_MD = (5, 1)
END_MD = (8, 31)

SPATIAL_STEP = 8  # 空间降采样步长
N_MODES = 6       #哪怕你只看前3个，算的时候多存几个没坏处

# --- Helper Functions ---
def get_file_path(date_obj):
    indir = date_obj.strftime('%d%b%y').lower() + '/'
    fname = date_obj.strftime('%Y%m%d')
    return os.path.join(BASE_DIR, indir, FNAME_HEAD + fname + '_' + fname + FNAME_TAIL)

def load_daily_sss(date_obj, step):
    """Loads, subsamples, and cleans one day of data."""
    path = get_file_path(date_obj)
    if not os.path.exists(path):
        return None, None, None, None
    try:
        with nc.Dataset(path, 'r') as f:
            # Load SSS (Time=0, Depth=0)
            sss = f.variables['vosaline'][0, 0, ::step, ::step]
            lon = f.variables['nav_lon'][::step, ::step]
            lat = f.variables['nav_lat'][::step, ::step]
            
            # Masking: Remove (0,0) artifacts and NaNs
            # SalishSeaCast model uses 0 for land in coords sometimes, or masked arrays
            mask = (lon != 0) & (~np.isnan(sss)) & (sss < 100) # Simple validity check
            
            return sss.flatten()[mask.flatten()], lon, lat, mask.flatten()
    except:
        return None, None, None, None

# --- Main Processing ---
if __name__ == "__main__":
    print(f"=== Starting Decade Analysis (2010-2019) ===")
    
    data_matrix = []
    time_index = []
    
    # Grid metadata (captured from the first valid file)
    grid_lon, grid_lat, grid_mask = None, None, None
    original_shape = None
    
    # 1. Loop through all dates
    for year in YEARS:
        d_start = datetime.date(year, *START_MD)
        d_end = datetime.date(year, *END_MD)
        curr = d_start
        print(f"  Loading year {year}...", end='\r')
        
        while curr <= d_end:
            vec, lon, lat, mask = load_daily_sss(curr, SPATIAL_STEP)
            
            if vec is not None:
                if grid_lon is None:
                    grid_lon, grid_lat, grid_mask = lon, lat, mask
                    original_shape = lon.shape
                
                # Double check grid consistency
                if len(vec) == grid_mask.sum():
                    data_matrix.append(vec)
                    time_index.append(curr)
            
            curr += datetime.timedelta(days=1)
    
    print(f"\nData Loading Done. Total Samples: {len(data_matrix)}")
    
    # 2. PCA Calculation
    X = np.array(data_matrix)
    print(f"Matrix Shape: {X.shape}")
    
    # De-mean (Anomaly)
    X_mean = X.mean(axis=0)
    X_anom = X - X_mean
    
    print("Computing PCA...")
    pca = PCA(n_components=N_MODES)
    PCs = pca.fit_transform(X_anom)   # (Time, Modes)
    EOFs = pca.components_            # (Modes, Space)
    Var = pca.explained_variance_ratio_
    
    # --- Sign Correction (Important for 10-year consistency) ---
    # Ensure the maximum amplitude of the spatial pattern is positive
    print("Standardizing signs...")
    for i in range(N_MODES):
        max_idx = np.argmax(np.abs(EOFs[i]))
        if EOFs[i][max_idx] < 0:
            EOFs[i] *= -1
            PCs[:, i] *= -1
            
    # 3. Saving Results
    
    # A. Save Time Series to CSV (User Request)
    print("Saving CSV...")
    df_pcs = pd.DataFrame(PCs, columns=[f'PC_{i+1}' for i in range(N_MODES)])
    df_pcs['Date'] = time_index
    df_pcs['Year'] = [d.year for d in time_index]
    df_pcs['DOY'] = [d.timetuple().tm_yday for d in time_index] # Day of Year
    
    csv_path = os.path.join(OUT_DIR, 'PCA_Decade_TimeSeries_2010_2019.csv')
    df_pcs.to_csv(csv_path, index=False)
    
    # B. Save Matrices for Plotting (Pickle/Numpy)
    # This saves EVERYTHING needed to reconstruct the plots without raw data
    npy_path = os.path.join(OUT_DIR, 'PCA_Decade_Spatial_Data.npz')
    np.savez(npy_path, 
             eofs=EOFs, 
             variance=Var, 
             lon=grid_lon, 
             lat=grid_lat, 
             mask=grid_mask, 
             shape=original_shape,
             mean_state=X_mean)
    
    print(f"SUCCESS!\nCSV saved to: {csv_path}\nPlotting Data saved to: {npy_path}")

=== Starting Decade Analysis (2010-2019) ===
  Loading year 2019...
Data Loading Done. Total Samples: 1230
Matrix Shape: (1230, 2880)
Computing PCA...
Standardizing signs...
Saving CSV...
SUCCESS!
CSV saved to: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/PCA_Decade_TimeSeries_2010_2019.csv
Plotting Data saved to: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/PCA_Decade_Spatial_Data.npz


In [4]:
# SSS_PCA_Decade_Plot.py
# Reads the pre-calculated PCA results and generates high-quality plots.

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os

# --- Config ---
# 指向第一步生成的目录
IN_DIR = '/home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/'

CSV_PATH = os.path.join(IN_DIR, 'PCA_Decade_TimeSeries_2010_2019.csv')
NPY_PATH = os.path.join(IN_DIR, 'PCA_Decade_Spatial_Data.npz')

# --- 1. Load Data ---
print("Loading pre-computed data...")
data = np.load(NPY_PATH, allow_pickle=True)
df_pcs = pd.read_csv(CSV_PATH, parse_dates=['Date'])

eofs = data['eofs']
var_ratio = data['variance']
lon = data['lon']
lat = data['lat']
mask = data['mask']     # 1D boolean mask
grid_shape = data['shape'] # Original 2D shape (y, x)

# --- 2. Plotting Functions ---

def plot_spatial_mode(mode_idx):
    """Plots the 2D spatial pattern of a specific mode."""
    # Reconstruct 2D Field from 1D clean vector
    eof_1d = eofs[mode_idx]
    
    # Create full grid filled with NaNs
    full_field = np.full(mask.shape, np.nan)
    # Fill in valid points
    full_field[mask] = eof_1d
    # Reshape to 2D
    eof_2d = full_field.reshape(grid_shape)
    
    # Plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator(central_longitude=-124))
    ax.set_extent([-125.5, -122, 48, 50.5], crs=ccrs.PlateCarree())
    
    # Adaptive Color Limits (Symmetric)
    limit = np.nanmax(np.abs(eof_2d))
    levels = np.linspace(-limit, limit, 41)
    
    cf = ax.contourf(lon, lat, eof_2d, levels=levels, cmap='RdBu_r', 
                     transform=ccrs.PlateCarree(), extend='both')
    
    ax.add_feature(cfeature.LAND, color='lightgray', zorder=2)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.8, zorder=3)
    ax.add_feature(cfeature.BORDERS, linestyle=':', zorder=3)
    
    cbar = plt.colorbar(cf, ax=ax, shrink=0.7, pad=0.05)
    cbar.set_label('EOF Amplitude')
    
    title = f"Mode {mode_idx+1} Spatial Pattern (Explains {var_ratio[mode_idx]*100:.1f}%)"
    ax.set_title(title, fontsize=14)
    
    save_name = os.path.join(IN_DIR, f'Decade_EOF_Mode_{mode_idx+1}.svg')
    plt.savefig(save_name, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved Map: {save_name}")

def plot_time_series_10years(mode_idx):
    """Plots the full 10-year time series."""
    pc_col = f'PC_{mode_idx+1}'
    
    plt.figure(figsize=(15, 4))
    
    # Plot the line
    plt.plot(df_pcs['Date'], df_pcs[pc_col], 'k-', linewidth=0.8, alpha=0.8)
    
    # Highlight 2011 and 2018 (Your specific interest)
    # Masking for colors
    subset_2011 = df_pcs[df_pcs['Year'] == 2011]
    plt.plot(subset_2011['Date'], subset_2011[pc_col], 'r-', linewidth=1.5, label='2011 (Wet/High Flow)')
    
    subset_2018 = df_pcs[df_pcs['Year'] == 2018]
    plt.plot(subset_2018['Date'], subset_2018[pc_col], 'b-', linewidth=1.5, label='2018 (Dry/Low Flow)')
    
    plt.ylabel('PC Amplitude')
    plt.title(f"Mode {mode_idx+1} Time Series (2010-2019)", fontsize=14)
    plt.grid(True, linestyle=':', alpha=0.5)
    plt.legend()
    
    save_name = os.path.join(IN_DIR, f'Decade_PC_{mode_idx+1}_TimeSeries.svg')
    plt.savefig(save_name, dpi=200, bbox_inches='tight')
    plt.close()
    print(f"Saved TS: {save_name}")

def plot_doy_overlay(mode_idx):
    """Overlays all years on a Day-of-Year axis to see seasonal consistency."""
    pc_col = f'PC_{mode_idx+1}'
    
    plt.figure(figsize=(12, 5))
    
    # Plot background years (Gray)
    for year in df_pcs['Year'].unique():
        subset = df_pcs[df_pcs['Year'] == year]
        if year not in [2011, 2018]:
            plt.plot(subset['DOY'], subset[pc_col], color='gray', alpha=0.3, linewidth=1)
            
    # Highlight specific years
    s2011 = df_pcs[df_pcs['Year'] == 2011]
    plt.plot(s2011['DOY'], s2011[pc_col], 'r-', linewidth=2, label='2011')
    
    s2018 = df_pcs[df_pcs['Year'] == 2018]
    plt.plot(s2018['DOY'], s2018[pc_col], 'b--', linewidth=2, label='2018')
    
    plt.xlabel('Day of Year')
    plt.ylabel('PC Amplitude')
    plt.title(f"Mode {mode_idx+1}: Seasonal Variability (Overlaid)", fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.5)
    
    save_name = os.path.join(IN_DIR, f'Decade_PC_{mode_idx+1}_Seasonality.svg')
    plt.savefig(save_name, dpi=200, bbox_inches='tight')
    plt.close()
    print(f"Saved Seasonal Plot: {save_name}")

# --- 3. Execute Plotting ---
if __name__ == "__main__":
    # Plot top 3 modes
    for i in range(3):
        print(f"--- Plotting Mode {i+1} ---")
        plot_spatial_mode(i)
        plot_time_series_10years(i)
        plot_doy_overlay(i)
        
    print("All plots generated.")

Loading pre-computed data...
--- Plotting Mode 1 ---
Saved Map: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/Decade_EOF_Mode_1.svg
Saved TS: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/Decade_PC_1_TimeSeries.svg
Saved Seasonal Plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/Decade_PC_1_Seasonality.svg
--- Plotting Mode 2 ---
Saved Map: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/Decade_EOF_Mode_2.svg
Saved TS: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/Decade_PC_2_TimeSeries.svg
Saved Seasonal Plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/Decade_PC_2_Seasonality.svg
--- Plotting Mode 3 ---
Saved Map: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_PCA_10yr/Decade_EOF_Mo

## Method Script

In detail, we can say:

To investigate the spatiotemporal variability of the Fraser River plume and its response to wind forcing, we performed an Empirical Orthogonal Function (EOF) analysis, also known as Principal Component Analysis (PCA), on the model-simulated Sea Surface Salinity (SSS) fields.

The analysis utilizes daily-averaged SSS output from the SalishSeaCast model (version 201905) covering a ten-year period from 2010 to 2019. For each year, we extracted data for the extended summer season, defined as May 1st to August 31st, to capture the full progression of the Fraser River freshet and the dominant summer wind patterns.

Data Preprocessing: To focus on synoptic-scale variability (e.g., wind-driven plume advection) while filtering out high-frequency tidal oscillations, daily averaged fields were selected. Spatially, the model grid was subsampled by a factor of 8 in both zonal and meridional directions to reduce computational cost while preserving mesoscale features. Land points and model boundary artifacts were masked out.

PCA Decomposition: The 3D SSS data (time, latitude, longitude) were flattened into a 2D matrix X(t,s), where t represents the daily time steps (N=1230 days total) and s represents the spatial grid points. The temporal mean at each grid point was subtracted to obtain the SSS anomaly field X 
′
 . The PCA was then performed on the covariance matrix of X 
′
 to decompose the variability into orthogonal spatial modes (EOFs) and their corresponding temporal coefficients (Principal Components, PCs).

The first few modes were retained for analysis, as they represent the dominant coherent patterns of salinity variability in the Salish Sea. The sign of the EOFs was standardized such that the maximum absolute amplitude is positive, ensuring consistency across different modes.

But for short:

Principal Component Analysis (PCA) of Summer SSS (2010–2019). We applied PCA to daily-averaged Sea Surface Salinity (SSS) anomalies from the SalishSeaCast model to identify dominant spatiotemporal patterns. The dataset covers the summer months (May 1 – Aug 31) over a decadal period (2010–2019). Daily averages were used to resolve synoptic wind events while removing tidal signals. The spatial grid was subsampled by a factor of 8. The data were centered by subtracting the long-term mean prior to decomposition. The resulting spatial modes (EOFs) and time series (PCs) describe the deviation of the plume structure from the mean state.

## Results Script

Results: Spatiotemporal Variability of Summer Sea Surface Salinity

The Principal Component Analysis (PCA) applied to the daily sea surface salinity (SSS) anomalies for the summers (May–August) of 2010–2019 reveals three dominant modes of variability. Together, these three modes explain approximately 67.5% of the total variance in the SSS field, capturing the primary drivers of the salinity structure in the Salish Sea: river discharge volume, wind-driven redistribution, and plume dispersion dynamics.

1. The Seasonal Discharge Mode (EOF1)

The first mode (EOF1) is the dominant component, accounting for 47.7% of the total variance (Fig. Xa). Spatially, this mode exhibits a coherent, monopole structure across the entire Strait of Georgia, with the highest amplitudes centered near the Fraser River mouth and extending into the central basin. The uniform sign of the spatial coefficients indicates that SSS variations associated with this mode occur in phase across the entire domain.

Physically, EOF1 represents the seasonal accumulation and draining of the total freshwater volume within the Strait. It captures the primary response to the Fraser River freshet: a strong positive phase in the associated Principal Component (PC1) corresponds to periods of high river discharge, resulting in a widespread freshening of the surface layer. Conversely, negative phases correspond to periods of reduced discharge or increased salinity prior to the freshet peak. This mode serves as a proxy for the total freshwater budget of the system.

2. The Wind-Driven Upwelling-Advection Dipole (EOF2)

The second mode (EOF2) explains 13.6% of the variance and reveals a distinct north-south dipole structure (Fig. Xb). The zero-crossing line (nodal line) is situated approximately near the Fraser River mouth, separating the domain into northern and southern sub-basins with opposing salinity anomalies.

Notably, the spatial morphology of this mode suggests a spatially coupled hydrodynamic response to the prevailing along-strait winds:

In the Northern Strait: The spatial pattern is boundary-intensified, with strong anomalies concentrated along the eastern and western shallow margins (e.g., Texada Island and Malaspina Strait). This structure is consistent with wind-driven coastal upwelling, where northwesterly winds drive surface Ekman transport, inducing the upwelling of saline deep water along the boundaries.

In the Southern Strait: The pattern is characterized by a broad, basin-wide anomaly extending towards the Gulf Islands and Boundary Pass. This reflects the meridional advection of the stratified Fraser River plume.

Under typical summer conditions (northwesterly winds), this mode manifests as a southward extension of the fresh plume (freshening the south) coupled with upwelling-favorable conditions in the north (increasing salinity). During reversal events (southeasterly winds), the pattern inverts, pushing the plume northward and suppressing the northern upwelling.

3. The Plume Dispersion Mode (EOF3)

The third mode (EOF3) accounts for 6.2% of the variance and captures a cross-channel contrast (Fig. Xc). The spatial pattern highlights a dynamic opposition between the "near-field" region (immediate vicinity of the river mouth and southern tidal channels) and the "far-field" region (central Strait and northern basin).

This mode represents the variability in plume retention versus dispersion. A positive phase (as shown in Fig. Xc, red near the mouth) indicates conditions where freshwater is confined close to the delta or trapped along the coast, likely due to weak mixing or specific tidal phases. A negative phase (blue near the mouth) suggests enhanced cross-channel spreading and effective mixing of freshwater into the ambient central basin. While explaining a smaller fraction of the total variance, this mode is significant for understanding the distinct plume geometry independent of the total discharge volume or wind direction.

Mode 1 (单极子): 

这是一个整体性的变化，不是哪里多了哪里少了，而是整个池子水变淡了或变咸了。这是为了对应 Fraser River 的总流量。

Mode 2: 

"spatially coupled hydrodynamic response"（空间耦合的水动力响应）。

North: "boundary-intensified"（边界强化）和 "consistent with wind-driven coastal upwelling"。“海峡两岸变色”的现象，暗示了涌升流机制。

South:  "meridional advection"（经向平流）。**(We will change the phase)** “一团水被推着走”的现象。

因为北方是 Upwelling 主导，所以营养盐主要由这个物理过程控制……

Mode 3 (扩散/滞留)：

"near-field vs. far-field"（近场 vs 远场）

解释为什么河口和海峡中间颜色相反。代表水是“堆在门口”还是“散开了”。